## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [35]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [36]:
import os
import pathlib

In [37]:
this_dir = pathlib.Path(os.path.abspath(""))

In [38]:
data_dir = this_dir / "data"

In [39]:
import pytz
import glob
import json
import requests
import pandas as pd
from datetime import datetime

## Download

Retrieve the page

In [40]:
url = "https://services3.arcgis.com/ibgDyuD2DLBge82s/arcgis/rest/services/Fresno_County_Zip_Data_Summary/FeatureServer/0/query?f=json&where=FZIP%3C%3E%2799999%27%20AND%20COUNT_Cases%3E%3D75%20AND%20FZIP%20IS%20NOT%20NULL&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&groupByFieldsForStatistics=FCITY&orderByFields=value%20desc&outStatistics=%5B%7B%22statisticType%22%3A%22sum%22%2C%22onStatisticField%22%3A%22COUNT_Cases%22%2C%22outStatisticFieldName%22%3A%22value%22%7D%5D&resultType=standard&cacheHint=true"

In [41]:
r = requests.get(url)

In [42]:
data = r.json()

## Parse

In [43]:
dict_list = []

In [44]:
for item in data["features"]:
    d = dict(
        county="Fresno",
        area=item["attributes"]["FCITY"],
        confirmed_cases=item["attributes"]["value"],
    )
    dict_list.append(d)

In [45]:
df = pd.DataFrame(dict_list)

In [46]:
df["area"] = df["area"].str.title()

Get timestamp

In [47]:
date_url = "https://services3.arcgis.com/ibgDyuD2DLBge82s/arcgis/rest/services/Fresno_County_Zip_Data_Summary/FeatureServer/0/?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [48]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [49]:
tz = pytz.timezone("America/Los_Angeles")

In [50]:
latest_date = datetime.fromtimestamp(timestamp / 1000, tz).date()

In [51]:
df["county_date"] = latest_date

## Trim

In [52]:
df["county"] = "Fresno"

In [53]:
df["area"] = df["area"].str.title()

In [54]:
c = df[["county", "area", "county_date", "confirmed_cases"]]

Remove county row

In [55]:
df = df[~(df.area == "Fresno")]

## Vet

In [56]:
try:
    assert not len(df) > 27
except AssertionError:
    raise AssertionError("Fresno County's place scraper has additional rows")

AssertionError: Fresno County's place scraper has additional rows

In [ ]:
try:
    assert not len(df) < 27
except AssertionError:
    raise AssertionError("Fresno County's place scraper is missing rows")

## Export

Mark the current date

In [ ]:
tz = pytz.timezone("America/Los_Angeles")

In [ ]:
today = datetime.now(tz).date()

In [ ]:
slug = "fresno"

In [ ]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [ ]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [ ]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [ ]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [ ]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)